In [10]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Initialize

In [37]:
def read_word2vec_text(vectors_fn, vocab_fn="", fmt="float64"):
    with open(vectors_fn) as f:
        file_pos = f.tell()
        V = np.loadtxt(f, dtype=str, comments=None, usecols=(0,), skiprows=1)
        word2index = {w: i for i, w in enumerate(V)}
        print(word2index)
        f.seek(file_pos)
        W = np.loadtxt(f, dtype=fmt, comments=None, skiprows=1, converters={0: lambda x: 0.0})
        if len(word2index) != len(W):
            print >>sys.stderr, "WARNING:", len(word2index),
            print >>sys.stderr, "disjoint words in embedding of length", len(W)
    return W[:, 1:], word2index

In [40]:
fn = '/home/eszti/projects/dipterv/notebooks/panlex/train_try/6_dog_cat.vec'
model = read_word2vec_text(fn)
model[0].shape[0]

{'dog': 0, 'kutya': 4, 'macska': 5, 'gatto': 9, 'gato': 7, 'hund': 2, 'perro': 6, 'cat': 1, 'katze': 3, 'cane': 8, 'chat': 11, 'chien': 10}


12

In [ ]:
def calc_precision(precs, trans, orig, trans_list, orig_list, trans_orig_dict):
    cos_mx = cosine_similarity(trans, orig)
    sim_mx = np.argsort(-cos_mx)
    max_prec = max(precs)
    prec_cnt = np.zeros(shape=(1, max_prec))
    print('word: \ttranslations in dict: \tclosest words after translation: \t')
    for i, r in enumerate(sim_mx):
        key_word = trans_list[i]
        value_words = trans_orig_dict[key_word]
        closest_words = []
        for j in range(max_prec):       
            ans = np.where(r==j)
            idx_orig = ans[0][0]
            word = orig_list[idx_orig]
            closest_words.append(word)
            if word in value_words:
                prec_cnt[0][j] = prec_cnt[0][j] + 1
        print('{}"\t{}\t{}'.format(word.encode('utf-8'), value_words, closest_words))
    print prec_cnt
    for i, val in enumerate(precs):
        sum_hit = np.sum(prec_cnt[0][0:val])
        print('prec {} : {}'.format(val, float(sum_hit)/sim_mx.shape[0]))

In [8]:
def train(langs, embs, d_idxs, d_wordpairs, dim, iters, lr=0.3):
    nb_langs = len(langs)

    # Init graphs
    graph = tf.Graph()
    with graph.as_default():
        # TF variables 
        # Placeholder for 2 words
        tf_w1 = tf.placeholder(tf.float64, shape=[None, dim])
        tf_w2 = tf.placeholder(tf.float64, shape=[None, dim])
        # Translation matrices
        tf_T = tf.Variable(tf.truncated_normal([nb_langs, dim, dim]))

        # Loss
        loss = tf.Variable([0], tf.float64)
        for (l1, l2), wps in d_wordpairs.items():
            for (w1, w2) in wps:
                idx_l1 = langs.index(l1)
                idx_l2 = langs.index(l2)
                idx_w1 = d_idx[l1][w1]
                idx_w2 = d_idx[l2][w2]
                # (Wi*Ti)*(Wj*Tj)'
                loss += tf.matmul(tf.matmul(tf_W_emb[idx_w1], tf_T[idx_l1]), 
                                  tf.transpose(tf.matmul(tf_W_emb[idx_w2], tf_T[idx_l2])))

        optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()

        for i in range(iters):
             # Run the computations
            _, l, T = session.run([optimizer, loss, tf_T])